# Text classification with TensorFlow Hub

Folowed this tutorial from the Tensorflow website 
https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

In [1]:
import tensorflow_hub as hub 

import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [14]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:1])

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ]],
      dtype=float32)>

In [17]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_7 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [19]:
 model.compile(optimizer='adam', 
               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
               metrics=['accuracy'])

In [20]:
history = model.fit(train_data.shuffle(1000).batch(512),
                   epochs=20,
                   validation_data=validation_data.batch(512),
                   verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 107ms/step - loss: 0.7030 - accuracy: 0.5585 - val_loss: 0.6594 - val_accuracy: 0.5964
Epoch 2/20
30/30 [==============================] - 2s 76ms/step - loss: 0.6336 - accuracy: 0.6213 - val_loss: 0.6155 - val_accuracy: 0.6413
Epoch 3/20
30/30 [==============================] - 2s 75ms/step - loss: 0.5961 - accuracy: 0.6579 - val_loss: 0.5831 - val_accuracy: 0.6680
Epoch 4/20
30/30 [==============================] - 2s 75ms/step - loss: 0.5592 - accuracy: 0.6853 - val_loss: 0.5505 - val_accuracy: 0.6858
Epoch 5/20
30/30 [==============================] - 2s 76ms/step - loss: 0.5223 - accuracy: 0.7185 - val_loss: 0.5157 - val_accuracy: 0.7290
Epoch 6/20
30/30 [==============================] - 2s 77ms/step - loss: 0.4810 - accuracy: 0.7532 - val_loss: 0.4806 - val_accuracy: 0.7470
Epoch 7/20
30/30 [==============================] - 2s 76ms/step - loss: 0.4423 - accuracy: 0.7820 - val_loss: 0.4474 - val_accuracy: 0.7776
Epoch 8/20
3

In [21]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))


loss: 0.318
accuracy: 0.859
